In [1]:
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from model.pipeline import SongPipeline
from model.utils import clean

In [2]:
def top_k_emotions(data:list, threshold:float, k:int):
    texts = data['lyric']
    titles = data['title']
    singers = data['singer']
    
    if not 0 <= threshold <=1:
        raise ValueError("theshold must be a float b/w 0 ~ 1.")
    results = {}
    for idx, text in enumerate(texts):
        cur_result = {}
        text = clean(text)
        for out in pipe(text, stride=128, return_overflowing_tokens=True,
                        padding=True, truncation=True)[0]:
            if out["score"] > threshold:
                cur_result[out["label"]] = round(out["score"], 2)
        cur_result = sorted(cur_result.items(), key=lambda x: x[1], reverse=True)
        title_plus_singer = titles[idx] + ' | ' + singers[idx]
        results[title_plus_singer] = cur_result[:k]
        
    return results


In [3]:
lyrics = pd.read_csv("New_Lyrics.csv")
model_name = "searle-j/kote_for_easygoing_people"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [4]:

pipe = SongPipeline(
        model=model,
        tokenizer=tokenizer,
        device=0, # gpu number, -1 if cpu used
        return_all_scores=True,
        function_to_apply='sigmoid'
    )


In [5]:
outputs = top_k_emotions(data=lyrics, threshold=0.2, k=5)
with open('results_new_pipeline.pkl', 'wb') as f:
    pickle.dump(outputs, f)
print('Finished!')

/opt/ml/final_project/develop/.venv/lib/python3.8/site-packages/transformers/pipelines/base.py:996: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Finished!


In [6]:
with open('results.pkl', 'rb') as f:
    result_original = pickle.load(f)

with open('results_new_pipeline.pkl', 'rb') as f:
    result = pickle.load(f)

In [7]:
for idx, key in enumerate(result_original.keys()):
    print(key)
    print("단순히 앞에서 512 글자수로 자름 : ", result_original[key])
    print("stride를 통해 나온 점수의 평균 : ", result[key])
    print('-' * 50)
    if idx == 10:
        break

Coming Back | 싸이 (PSY)
단순히 앞에서 512 글자수로 자름 :  [('즐거움/신남', 0.94), ('기쁨', 0.87), ('행복', 0.84), ('기대감', 0.73), ('환영/호의', 0.47)]
stride를 통해 나온 점수의 평균 :  [('즐거움/신남', 0.65), ('없음', 0.59), ('기대감', 0.51), ('기쁨', 0.43), ('행복', 0.35)]
--------------------------------------------------
TOMBOY | (여자)아이들
단순히 앞에서 512 글자수로 자름 :  [('화남/분노', 0.62), ('짜증', 0.62), ('비장함', 0.55), ('없음', 0.5), ('어이없음', 0.39)]
stride를 통해 나온 점수의 평균 :  [('없음', 0.78), ('비장함', 0.32), ('화남/분노', 0.26), ('짜증', 0.24)]
--------------------------------------------------
LOVE DIVE | IVE (아이브)
단순히 앞에서 512 글자수로 자름 :  [('행복', 0.8), ('아껴주는', 0.78), ('즐거움/신남', 0.71), ('기대감', 0.7), ('기쁨', 0.7)]
stride를 통해 나온 점수의 평균 :  [('없음', 0.71), ('아껴주는', 0.58), ('행복', 0.4), ('기대감', 0.33), ('기쁨', 0.33)]
--------------------------------------------------
봄여름가을겨울 (Still Life) | BIGBANG (빅뱅)
단순히 앞에서 512 글자수로 자름 :  [('슬픔', 0.89), ('힘듦/지침', 0.72), ('안타까움/실망', 0.51), ('불쌍함/연민', 0.47), ('서러움', 0.45)]
stride를 통해 나온 점수의 평균 :  [('슬픔', 0.84), ('힘듦/지침', 0.68), ('아껴주

In [ ]:
백엔드 쪽에서는
처음에는 감정 44개 프로토타입을 만들고

감정 분류가 되면, 어떤 거 쓰고 안쓰고
코드에 넣으면 바로 될거 같거든요

44개

사용자가 요청한 감정 중에 저희가 정한 20개 내에 없으면? (0개면?)
0개면 => (무슨 감정인지 모르겠어요!) 체크리스트 보여주면 될 거 같습니다
data가 없어서 

저희 모델로 예측 한 다음에... 그 예측값으로 레이블링 하고? 수도 레이블링하고
거기서 다시 fine-tuning하고 그래야 하는데...

In [ ]:
논문에서 이 연구팀이 훈련시킨 방식이 그거거든요?
Binary cross entropy로 훈련을 시켰는데,
softmax가 아니라 sigmoid를 썻어요
softmax는 수식이 전체 레이블에 대해서 확률을 구하는 =>
sigmoid는 => independent해요 레이블끼리
threshold를 넘으면 1, 아니면 0